In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv("train.csv")
trainX = torch.from_numpy(df.iloc[:,:-1].values)
trainY = torch.from_numpy(df.iloc[:,-1:].values)
trainX = trainX.to(torch.float64)
trainY = trainY.to(torch.float64)

In [3]:
df = pd.read_csv("test.csv")
testX = torch.from_numpy(df.iloc[:,:-1].values)
testY = torch.from_numpy(df.iloc[:,-1:].values)
testX = testX.to(torch.float64)
testY = testY.to(torch.float64)

In [4]:
#Q1
...
trainX.nelement() * trainX.element_size()   # "about" how many bytes are used

83520

In [5]:
#Q2
...
trainX16 = trainX.to(torch.float16)
trainX16to64 = trainX16.to(torch.float64)
q2result = trainX16 - trainX16to64
torch.max(q2result).item()

0.0

In [6]:
#Q3
...
torch.cuda.is_available()

False

In [7]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [8]:
#Q4
...
firstRow = testX[:1,]@coef
firstRow.item()

9.844

In [9]:
#Q5
...
whole = testX@coef
torch.mean(whole).item()

12.073632183908048

In [10]:
def f(x):
    return x**2 - 8*x + 19

In [11]:
#Q6
...
x = torch.tensor(0.0)
y = f(x)
float(y)

19.0

In [12]:
import matplotlib.pyplot as plt

In [13]:
#Q7
...
x = torch.arange(-8, 5, 0.1)
y = f(x)

x = torch.tensor(0.0, requires_grad=True)
optimizer = torch.optim.SGD([x], lr=0.01)

for epoch in range(1_000):
    y = f(x)
    y.backward()
    optimizer.step()
    optimizer.zero_grad()
x.item()

3.9999940395355225

In [14]:
loss_fn = torch.nn.MSELoss()

In [15]:
#Q8
...
coef = torch.zeros((trainX.shape[1], 1), dtype=torch.float64, requires_grad=True)
prediction = trainX@coef
loss_fn(prediction, trainY).item()

197.8007662835249

In [16]:
# Seed torch random number generator with 544
torch.manual_seed(544)

In [17]:
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)

In [18]:
coef = torch.zeros((trainX.shape[1], 1), dtype=torch.float64, requires_grad=True)
optimizer = torch.optim.SGD([coef], lr=0.000002)

for epoch in range(500):
    for batchX, batchY in dl:
        predictions = batchX @ coef
        loss = loss_fn(predictions, batchY)    
        loss.backward() # computes gradient, and adds it to coef.grad
        optimizer.step()
        optimizer.zero_grad()

In [19]:
prediction = trainX@coef

In [20]:
#Q9
...
loss_fn(prediction, trainY).item()

26.8113940147193

In [21]:
#Q10
...
prediction_test = testX@coef
loss_fn(prediction_test, testY).item()

29.05854692548551